## Import Package

In [1]:
import pandas as pd
from pandas import DataFrame
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler

## Basic Function

In [2]:
def Day(s):
    d = s.split('/')
    dd = int(d[1])
    mm = int(d[0])
    d = dd + month_day_list[mm-1]
    return d

#mm/dd/yy-->number
month_day_list = [31,29,31,30,31,30,31,31,30,31,30,31]
for i in range(1,12):
    month_day_list[i] += month_day_list[i-1]

def M_Dict(c):
    c_key = []
    c_value = []
    for i in range(len(c)):
        c_key.append(c[i][0])
        c_value.append([])
        for j in range(len(c[0])-1):
            c_value[i].append(c[i][j+1])
    c_dict = dict(zip(c_key, c_value))
    return c_dict
   

##  Add Daily Report

In [3]:
df = pd.read_csv("us_counties_covid19_daily.csv")
df = df.dropna()

#drop place without local data as population)
df = df[df.fips != 69110.0]
df = df[df.fips != 69120.0]
df = df[df.fips != 78010.0]
df = df[df.fips != 78020.0]
df = df[df.fips != 78030.0]

data = df.values.tolist()

column_name_1 = ['date','county','state','fips','case','death','new_case']

for i in range(len(data)):
    data[i][3]=int(data[i][3])

##  Add Local Information

In [4]:
c = pd.read_csv("us_county.csv") 
c = c.values.tolist()

c_dict = M_Dict(c)
"""
c_key = []
c_value = []
for i in range(len(c)):
    c_key.append(c[i][0])
    c_value.append([])
    for j in range(len(c[0])-1):
        c_value[i].append(c[i][j+1])
c_dict = dict(zip(c_key, c_value))
"""

# first case data-->c_dict & add_new_case_number
fips = 1000
for i in range(len(data)):
    if data[i][3] != fips:
        c_dict[data[i][3]].append(data[i][0])
        fips = data[i][3]
        data[i].append(data[i][4])
    else:
        data[i].append(data[i][4]-data[i-1][4])

column_name_2 = ['state_init', 'male', 'female', 'median_age', 'population', 'female_percentage', 'lat', 'long', 'first_case_time']
#data add state_init,male, female, median_age, population, female_percentage, lat, long & first case time
#dict[x][3~7]
for i in range(len(data)):
    for j in range(2,11):
        data[i].append(c_dict[data[i][3]][j])
        
for i in range(len(data)):
    data[i][0] = Day(data[i][0])
    data[i][-1] = Day(data[i][-1])

## Add Currently Policy

In [5]:
column_name_3 = ['stay_at_home','50_gatherings' ,'500_gatherings','public_schools','restaurant','entertainment/gym','Federal_guidelines','foreign_travel_ban']
s = pd.read_csv("Action.csv") 
s = s.fillna(999)
s = s.values.tolist()

n = s[0][5]

for i in range(len(s)):
    for j in range(1,len(s[0])-2):
        if s[i][j] != 999:
            d = s[i][j].split('-')
            s[i][j] = month_day_list[2] + int(d[0])
    
s_dict = M_Dict(s)

for i in range(len(data)):
    for j in range(8):
        if data[i][0] > s_dict[data[i][7]][j]:
            data[i].append(1)
        else:
            data[i].append(0)

## Merge，Normailzation & Output

In [6]:
for i in range(len(data)):
    data[i].pop(7)
    data[i].pop(2)
    data[i].pop(1)

In [7]:
C_Name = column_name_1+column_name_2+column_name_3
C_Name.pop(7)
C_Name.pop(2)
C_Name.pop(1)

'county'

In [8]:
#print(len(C_Name),len(data[0]))
ndf = DataFrame(data,columns=C_Name)
ndf = ndf.drop('fips', axis=1)
ndf = ndf.drop('death', axis=1)
clist = list(ndf.columns)
clist_new = ['new_case','first_case_time','date','case','median_age','population','female_percentage','stay_at_home','50_gatherings','500_gatherings','public_schools','restaurant','entertainment/gym','Federal_guidelines','foreign_travel_ban','male','female','lat','long']
ndf = ndf[clist_new]

dataX = ndf.iloc[:,1:19]
dataY = ndf.iloc[:,0]

scale = StandardScaler(with_mean=True,with_std=True)
dataXS = scale.fit_transform(dataX)
dataXS = dataXS.tolist()
dataY = dataY.tolist()

ndata = []
for i in range(len(dataXS)):
    ndata.append([dataY[i]] + dataXS[i])

In [9]:
#Output
#ndataf = DataFrame(ndata,columns=clist_new)
ndf.to_csv('merged_dateset.csv', encoding='utf-8', index=False)
train_dataset =ndf.sample(frac=0.7, random_state=1)
test_dataset = ndf.drop(train_dataset.index)
train_dataset.to_csv('train_dataset.csv', encoding='utf-8', index=False)
test_dataset.to_csv('test_dataset.csv', encoding='utf-8', index=False)
train_dataset.head()

new_case  first_case_time  date   case  median_age  population  \
27747          2              123   198     98        36.2       42831   
517741         0              124   186      7        21.7        7110   
61114         68              107   266  10752        37.2      311522   
50298         -1              114   199     85        39.9       55101   
155987         3              110   121     13        38.1       42950   

        female_percentage  stay_at_home  50_gatherings  500_gatherings  \
27747           51.259602             0              0               0   
517741          45.007032             0              1               1   
61114           50.398367             1              1               1   
50298           49.565344             1              1               1   
155987          49.997672             0              0               0   

        public_schools  restaurant  entertainment/gym  Federal_guidelines  \
27747                1           1                  1                   1   
517741               1           1                  1                   1   
61114                1           1                  1                   1   
50298                1           1                  1                   1   
155987               0           1                  1                   1   

        foreign_travel_ban    male  female        lat        long  
27747                    1   20876   21955  35.763834  -90.054201  
517741                   1    3910    3200  37.782506  -79.443984  
61114                    1  154520  157002  30.668927  -87.362783  
50298                    1   27790   27311  37.286554 -107.843330  
155987                   1   21476   21474  41.483921  -91.112707